#HIDS-506 Final Project: One-Hot-Encoding Chief Complaints

## Packages



We will start by loading some of the packages that will help us 
organize and visualize the data. Other packages will be loaded as necessary.

In [ ]:
#Import packages
from collections import Counter
import pandas as pd
import numpy as np
import collections, numpy
import mlxtend
import matplotlib 
from matplotlib import pyplot 
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
matplotlib.rcParams['figure.figsize'] = (10, 10)


from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
from sklearn.metrics import roc_curve, precision_recall_curve, auc

## Dataset

**Research Question:** Can machine learning (ML) provide a data-driven alternative to current emergency care triage and produce an accurate patient acuity score based on minimal patient information upon admission to the Emergency Department (ED), thus reducing human-biased medical decisions and preserving clinician time, attention, and ED workflow? 

Triage_clean_df Dataframe:
Attempt 1 - Chief Complaint one-hot-encoding and rescaling of patient acuity score distribution

Note: This attempt was unsuccessful in rescaling patient acuity and resulted in a perfectly balanced dataset unfit for modeling an accuracte prediction.




Exploring the datasets:

In [ ]:
#from google.colab import 
#drivedrive.mount('/content/drive')

In [ ]:
#mounting google drive to access files 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Triage_clean_df

In [ ]:
# triage clean file includes cleaned dataframe with all merged features from MIMIC-IV
# defining triage_clean_file path 
triage_clean_file = '/content/drive/MyDrive/HIDS_506_FinalProject/data/triage_clean.csv'

# read in triage_clean_file to dataframe
triage_clean_df = pd.read_csv(triage_clean_file)

In [ ]:
# visualizing the first few rows of triage_clean_df.csv
triage_clean_df.head()

Unnamed: 0  subject_id  temperature  heartrate  resprate  o2sat   sbp  \
0           0    10000032         97.8       87.0      14.0   97.0  71.0   
1           1    10000032         97.8       87.0      14.0   97.0  71.0   
2           2    10000032         97.8       87.0      14.0   97.0  71.0   
3           3    10000032         97.8       87.0      14.0   97.0  71.0   
4           4    10000032         97.8       87.0      14.0   97.0  71.0   

    dbp  pain  acuity  ... disposition_ELOPED disposition_EXPIRED  \
0  43.0     7     2.0  ...                  0                   0   
1  43.0     7     2.0  ...                  0                   0   
2  43.0     7     2.0  ...                  0                   0   
3  43.0     7     2.0  ...                  0                   0   
4  43.0     7     2.0  ...                  0                   0   

  disposition_HOME  disposition_LEFT AGAINST MEDICAL ADVICE  \
0                0                                        0   
1                0                                        0   
2                0                                        0   
3                1                                        0   
4                0                                        0   

   disposition_LEFT WITHOUT BEING SEEN  disposition_OTHER  \
0                                    0                  0   
1                                    0                  0   
2                                    0                  0   
3                                    0                  0   
4                                    0                  0   

   disposition_TRANSFER  age_category_middle_adult  age_category_senior  \
0                     0                          1                    0   
1                     0                          1                    0   
2                     0                          1                    0   
3                     0                          1                    0   
4                     0                          1                    0   

   age_category_young_adult  
0                         0  
1                         0  
2                         0  
3                         0  
4                         0  

[5 rows x 40 columns]

In [ ]:
triage_clean_df.columns

Index(['Unnamed: 0', 'subject_id', 'temperature', 'heartrate', 'resprate',
       'o2sat', 'sbp', 'dbp', 'pain', 'acuity', 'chiefcomplaint', 'intime',
       'outtime', 'gender', 'deceased', 'race_AMERICAN INDIAN/ALASKA NATIVE',
       'race_ASIAN', 'race_BLACK/AFRICAN AMERICAN', 'race_HISPANIC/LATINO',
       'race_NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER', 'race_OTHER/UNKNOWN',
       'race_PORTUGUESE', 'race_SOUTH AMERICAN', 'race_WHITE',
       'arrival_transport_AMBULANCE', 'arrival_transport_HELICOPTER',
       'arrival_transport_OTHER', 'arrival_transport_UNKNOWN',
       'arrival_transport_WALK IN', 'disposition_ADMITTED',
       'disposition_ELOPED', 'disposition_EXPIRED', 'disposition_HOME',
       'disposition_LEFT AGAINST MEDICAL ADVICE',
       'disposition_LEFT WITHOUT BEING SEEN', 'disposition_OTHER',
       'disposition_TRANSFER', 'age_category_middle_adult',
       'age_category_senior', 'age_category_young_adult'],
      dtype='object')

In [ ]:
# redefining column names
triage_clean_df = triage_clean_df[['subject_id', 'temperature', 'heartrate', 'resprate',
       'o2sat', 'sbp', 'dbp', 'pain', 'acuity', 'chiefcomplaint', 'intime',
       'outtime', 'gender', 'deceased', 'race_AMERICAN INDIAN/ALASKA NATIVE',
       'race_ASIAN', 'race_BLACK/AFRICAN AMERICAN', 'race_HISPANIC/LATINO',
       'race_NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER', 'race_OTHER/UNKNOWN',
       'race_PORTUGUESE', 'race_SOUTH AMERICAN', 'race_WHITE',
       'arrival_transport_AMBULANCE', 'arrival_transport_HELICOPTER',
       'arrival_transport_OTHER', 'arrival_transport_UNKNOWN',
       'arrival_transport_WALK IN', 'disposition_ADMITTED',
       'disposition_ELOPED', 'disposition_EXPIRED', 'disposition_HOME',
       'disposition_LEFT AGAINST MEDICAL ADVICE',
       'disposition_LEFT WITHOUT BEING SEEN', 'disposition_OTHER',
       'disposition_TRANSFER', 'age_category_middle_adult',
       'age_category_senior', 'age_category_young_adult']]

In [ ]:
triage_clean_df.head()

subject_id  temperature  heartrate  resprate  o2sat   sbp   dbp  pain  \
0    10000032         97.8       87.0      14.0   97.0  71.0  43.0     7   
1    10000032         97.8       87.0      14.0   97.0  71.0  43.0     7   
2    10000032         97.8       87.0      14.0   97.0  71.0  43.0     7   
3    10000032         97.8       87.0      14.0   97.0  71.0  43.0     7   
4    10000032         97.8       87.0      14.0   97.0  71.0  43.0     7   

   acuity chiefcomplaint  ... disposition_ELOPED disposition_EXPIRED  \
0     2.0    hypotension  ...                  0                   0   
1     2.0    hypotension  ...                  0                   0   
2     2.0    hypotension  ...                  0                   0   
3     2.0    hypotension  ...                  0                   0   
4     2.0    hypotension  ...                  0                   0   

   disposition_HOME  disposition_LEFT AGAINST MEDICAL ADVICE  \
0                 0                                        0   
1                 0                                        0   
2                 0                                        0   
3                 1                                        0   
4                 0                                        0   

   disposition_LEFT WITHOUT BEING SEEN  disposition_OTHER  \
0                                    0                  0   
1                                    0                  0   
2                                    0                  0   
3                                    0                  0   
4                                    0                  0   

   disposition_TRANSFER  age_category_middle_adult  age_category_senior  \
0                     0                          1                    0   
1                     0                          1                    0   
2                     0                          1                    0   
3                     0                          1                    0   
4                     0                          1                    0   

   age_category_young_adult  
0                         0  
1                         0  
2                         0  
3                         0  
4                         0  

[5 rows x 39 columns]

In [ ]:
# (2984649 observations, 39 features) prior to one-hot-encoding
triage_clean_df.shape

(2984649, 39)

### Re-scaling patient acuity score distribution 

In [ ]:
# patient acuity score distribution in dataset; imbalanced 
triage_clean_df['acuity'].value_counts()

3.0    1776350
2.0     930618
4.0     181285
1.0      87155
5.0       9241
Name: acuity, dtype: int64

In [ ]:
# Get the count of '5.0' category
count = triage_clean_df['acuity'].value_counts()[5.0]

# Group the dataframe by 'acuity' column and sample each group to the count of '5.0' category
triage_balanced_df = triage_clean_df.groupby('acuity').apply(lambda x: x.sample(n=count, random_state=42)).reset_index(drop=True)

In [ ]:
# shape of balaned dataframe from undersampling method above 
triage_balanced_df.shape

(46205, 39)

In [ ]:
# visualizing new acuity score distribution
triage_balanced_df['acuity'].value_counts()

1.0    9241
2.0    9241
3.0    9241
4.0    9241
5.0    9241
Name: acuity, dtype: int64

### One-hot-encoding Chief Complaints

In [ ]:
# visualizing count of chief complaint values 
triage_balanced_df['chiefcomplaint'].value_counts().sum()

46201

In [ ]:
Counter(triage_balanced_df['chiefcomplaint'])

Counter({'syncope': 126,
         'valium ingestion': 5,
         'si, overdose': 16,
         'l weakness, slurred speech': 4,
         'dizziness, hypotension, syncope': 2,
         'hypoxia': 95,
         'dyspnea': 1104,
         'fever': 325,
         'etoh, hypoglycemia': 48,
         'nausea': 39,
         'hypotension': 229,
         'altered mental status, transfer': 4,
         'word finding difficulty': 4,
         'abd pain/abnl ekg': 8,
         'bradycardia': 60,
         'dizziness, syncope': 6,
         'hematuria': 145,
         's/p fall': 413,
         'dizziness': 297,
         'abd pain': 1060,
         'dizzy': 28,
         'palpitations': 171,
         'hypotension, tachycardia': 3,
         'chest pain': 1230,
         'l weakness': 47,
         'sore throat, dyspnea on exertion, cough': 9,
         'hyperglycemia': 151,
         'seizure': 182,
         'abnormal sodium level': 7,
         'visual changes, weakness': 7,
         'slurred speech': 83,
         '

In [ ]:
# setting dummies to chief complaint and creating new dataframe of one-hot-encoded values
dummies = triage_balanced_df['chiefcomplaint'].str.get_dummies(sep=",")

In [ ]:
# concatenating dummies on triage dataframe
df_merged = pd.concat([triage_balanced_df, dummies], axis=1)

In [ ]:
# drop the original column used to create dummy variables
df_merged.drop(['chiefcomplaint'], axis=1, inplace=True)

In [ ]:
df_merged.head()

subject_id  temperature  heartrate  resprate  o2sat    sbp   dbp  pain  \
0    10731577         98.0       59.0      18.0  100.0   82.0  52.0     0   
1    13383991         97.2      142.0      16.0   98.0  114.0  82.0     0   
2    13264941         99.0      130.0      12.0   98.0  127.0  83.0     0   
3    16016920         99.8      102.0      16.0  100.0   89.0  68.0     9   
4    18289359         98.4      100.0      16.0   98.0   81.0  50.0     0   

   acuity               intime  ... wound discharge/redness/nausea  \
0     1.0  2123-01-07 12:33:00  ...                              0   
1     1.0  2168-12-22 16:40:00  ...                              0   
2     1.0  2128-09-11 09:01:00  ...                              0   
3     1.0  2151-11-03 06:10:00  ...                              0   
4     1.0  2147-05-17 09:18:00  ...                              0   

   wound eval  wound infection  wound infection right axilla  wound infx  \
0           0                0                             0           0   
1           0                0                             0           0   
2           0                0                             0           0   
3           0                0                             0           0   
4           0                0                             0           0   

   wound re-eval  wrist injury bilat  wrist pain  wt gain    
0              0                   0           0        0   0  
1              0                   0           0        0   0  
2              0                   0           0        0   0  
3              0                   0           0        0   0  
4              0                   0           0        0   0  

[5 rows x 3852 columns]

In [ ]:
df_merged.columns

Index(['subject_id', 'temperature', 'heartrate', 'resprate', 'o2sat', 'sbp',
       'dbp', 'pain', 'acuity', 'intime',
       ...
       'wound discharge/redness/nausea', 'wound eval', 'wound infection',
       'wound infection right axilla', 'wound infx', 'wound re-eval',
       'wrist injury bilat', 'wrist pain', 'wt gain', ''],
      dtype='object', length=3852)

In [ ]:
# save merged dataframe with one-hot-encoded chief complaints to csv file 
df_merged.to_csv('/content/drive/MyDrive/HIDS_506_FinalProject/data/df_merged_balanced.csv')